In [9]:
# Import the necessary libraries

from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import os
import sendgrid
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [2]:
# Load environment variables

load_dotenv(override=True)

True

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool.

Costs are here: https://platform.openai.com/docs/pricing#web-search

In [5]:
# Create the search agent

INSTRUCTIONS = "You are an experienced research assistant. Given a search term, you can easily and deftly search the web for that term and \
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 500 \
words. Capture the main points and be as thorough and insightful as possible. Write succintly and crisply, no need to have complete sentences or good \
grammar. Your tone is formal, professional, and authoritative. This will be consumed by someone synthesizing a report, \
so it's vital you capture the essence and ignore any fluff. Do not include any additional commentary other than the summary itself.\
Do not use any flowery language or any other words that are not necessary."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"), # This is important to ensure that the agent must use the tools. Mandatory usage!
    )

In [6]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of July 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent, autonomous systems.

**AutoGen**: An open-source framework by Microsoft, AutoGen facilitates the orchestration of multiple AI agents, enabling both autonomous and semi-autonomous workflows. It allows for dynamic collaboration among agents, with human intervention when necessary, making it suitable for tasks like collaborative problem-solving and code generation. However, its complexity may present a steep learning curve for newcomers. ([medium.com](https://medium.com/bridgged-generative-ai-strategy-hub/the-top-5-ai-agent-frameworks-to-watch-in-2025-bridgged-a9092f095d32?utm_source=openai))

**CrewAI**: This lightweight Python framework emphasizes role-based agent design, inspired by human team dynamics. CrewAI enables the creation of agents with distinct roles and capabilities, fostering effective collaboration. It's ideal for workflows requiring structured teamwork, such as marketing strategy automation and project management. While powerful, it may be complex for non-technical users. ([medium.com](https://medium.com/bridgged-generative-ai-strategy-hub/the-top-5-ai-agent-frameworks-to-watch-in-2025-bridgged-a9092f095d32?utm_source=openai))

**LangGraph**: Built upon LangChain, LangGraph offers a stateful execution environment for complex, multi-step workflows. Its ability to maintain state throughout a process is beneficial for applications needing persistence and error recovery, such as customer support systems and fraud detection. The framework's graph-based architecture supports dynamic decision-making and agent interactions. ([medium.com](https://medium.com/bridgged-generative-ai-strategy-hub/the-top-5-ai-agent-frameworks-to-watch-in-2025-bridgged-a9092f095d32?utm_source=openai))

**LlamaIndex**: An open-source data orchestration framework, LlamaIndex is designed for developing generative AI and AI agent solutions. It features ready-to-use agents and tools, along with workflows that enable the development of multi-agent systems. LlamaIndex streamlines context augmentation for generative AI use cases by utilizing a Retrieval-Augmented Generation (RAG) pipeline, integrating multiple tools and functionalities to simplify the development process. ([antiersolutions.com](https://www.antiersolutions.com/blogs/top-ai-agent-frameworks-to-watch-in-2025-a-complete-guide/?utm_source=openai))

**Eliza**: A Web3-friendly AI agent operating system, Eliza integrates seamlessly with web3 applications, allowing for the deployment of AI agents that interact with blockchain data and smart contracts. It emphasizes user control and stability, providing a robust platform for developing decentralized AI applications. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

**AutoAgent**: A fully-automated, zero-code framework for Large Language Model (LLM) agents, AutoAgent enables users to create and deploy LLM agents through natural language alone. It operates as an autonomous agent operating system, comprising components like Agentic System Utilities and a Self-Managing File System, facilitating efficient and dynamic creation and modification of tools, agents, and workflows without coding requirements. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

**AgentLite**: A lightweight library for building and advancing task-oriented LLM agent systems, AgentLite simplifies the process of creating and evaluating new reasoning strategies and agent architectures. It offers a user-friendly platform for innovating LLM agent reasoning, architectures, and applications, enhancing the ability of agents to break down tasks and facilitating the development of multi-agent systems. ([arxiv.org](https://arxiv.org/abs/2402.15538?utm_source=openai))

These frameworks represent the forefront of AI agent development, each contributing to the evolution of intelligent, autonomous systems across various applications. 

### Trace available at

https://platform.openai.com/traces

In [12]:
# Now lets create a planner agent and use more of structured outputs

HOW_MANY_SEARCHES = 3

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response 

class WebSearchItem(BaseModel):
    reason: str = Field(
        description="Your reasoning for why this search is important to the query."
    )

    query: str = Field(
        description="The search term to use for the web search."
    )


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(
        description="A list of web searches to perform to best answer the query."
    )


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan, # Force the output to be a WebSearchPlan instead of markdown
)

In [18]:
message = "Latest AI Agent frameworks in 2025"

with trace("Deep Research Agent"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find updated information on AI agent frameworks released or popular in 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To gather insights from industry reports or tech articles discussing trends in AI agent frameworks for 2025.', query='AI agent frameworks trends 2025'), WebSearchItem(reason='To explore specific frameworks that have gained traction among developers and researchers in 2025.', query='popular AI agent frameworks 2025')]


In [21]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("gpallapa@pm.me")  # Change to your verified sender
    to_email = To("gautham.pallapa@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [23]:
# Create the email agent

INSTRUCTIONS = """You are able to send a nicely formatted HTML email based on a detailed report.
You will be provided with a detailed report. You should use your tool to send one email, providing the 
report converted into clean, well presented HTML with an appropriate subject line."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)

In [25]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [26]:
# Plan and execute the search

async def plan_search(search_query: str):
    """ Use the planner agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {search_query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each search in the search plan """
    print("Performing searches...")
    num_completed = 0
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print(f"Completed {num_completed} searches")
    return results

async def search(search_item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {search_item.query}\n\nReason for searching: {search_item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [27]:
# These two functions will write a report based on the search results and email it to the user

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results """
    print("Thinking and compiling the report...")
    input = f"Original query: {query}\nSummarized results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing the report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to email the report to the user """
    print("Emailing report...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report

In [28]:
query = "Latest Ai Agent frameworks in 2025"

with trace("Research trace"):
    print(f"Query provided: {query}")
    print("Starting research...")
    search_plan = await plan_search(query)

    search_results = await perform_searches(search_plan)

    report = await write_report(query, search_results)

    await send_email(report)

    print("Research complete")

Query provided: Latest Ai Agent frameworks in 2025
Starting research...
Planning searches...
Will perform 3 searches
Performing searches...
Completed 0 searches
Thinking and compiling the report...
Finished writing the report
Emailing report...
Email sent
Research complete
